# Data Visualization 2.0

This notebook has some actual thinking going on in the data visualization

The important bit is the very last cell, everything prior is more of "how to call the helper functions"

Run the following cells:
1. Import Packages 
2. Helper Functions
3. Load Data File

## Import Packages

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import trackml

from trackml.dataset import load_event

import random


import plotly
import plotly.offline as offline
offline.init_notebook_mode(connected=True)

import plotly.plotly as py
import plotly.graph_objs as go

import plotly.tools as tls
% matplotlib inline

print ('Complete')

Complete


## Helper Functions

In [9]:
def find_nearby(df,x,y,z):
    '''
    find_nearby(df,x,y,z):
    
    Input: Hits Dataframe (df), Cartesian coordinates x,y,z in mm
    Return: Dataframe of points within the specified z and xy search distances
    '''
    zLook = 5 # mm
    xyLook = 10 #mm
    
    # Trim according to z value
    zDistance = np.abs(df.z-z)
    zTrimDF = dfH.loc[ zDistance < zLook]

    # trim according to y value
    yDistance = np.abs(zTrimDF.y-y)
    yTrimDF = zTrimDF.loc[ yDistance < xyLook ]

    # trim according to x value
    xDistance = np.abs(yTrimDF.x-x)
    xTrimDF = yTrimDF.loc[ xDistance < xyLook]
    return (xTrimDF)


def plot_trajectory(row):
    part_id = dfP.loc[row,'particle_id']
    assert (part_id != 0), 'Hit due to noise, no associated track'
    pdf = dfT.loc[ dfT.particle_id == part_id ]

    # Grab hits of particle
    xActual = np.asarray(pdf.tx)
    yActual = np.asarray(pdf.ty)
    zActual = np.asarray(pdf.tz)

#     print ('Particle ID: {0}'.format(part_id))
#     print ('Number of Hits: {0}'.format(len(xActual)))

    ##############################
    # Plot the particle trajectory
    ##############################
    trace1 = go.Scatter3d(
        x = xActual,
        y = yActual,
        z = zActual,
        mode = 'lines+markers',
        name = 'Trajectory',
        marker=dict(size=4,
                   color = 'blue'),
    )
    return (trace1)

def plot_starting(row):
    part_id = dfP.loc[row,'particle_id']
    assert (part_id != 0), 'Hit due to noise, no associated track'
    pdf = dfT.loc[ dfT.particle_id == part_id ]

    # Grab unique hit_ids and momenta of particle
    hit_ids = np.asarray(pdf.hit_id)
    xActual = np.asarray(pdf.tx)
    yActual = np.asarray(pdf.ty)
    zActual = np.asarray(pdf.tz)
    # Mark the starting point
    trace1 = go.Scatter3d(
        x = xActual[0:1],
        y = yActual[0:1],
        z = zActual[0:1],
        mode = 'markers',
        name = 'Start',
        marker = dict(
            size = 6,
            color = 'red'),
    )

    return (trace1)
    

def plot_nearby(row):
    part_id = dfP.loc[row,'particle_id']
    assert (part_id != 0), 'Hit due to noise, no associated track'
    pdf = dfT.loc[ dfT.particle_id == part_id ]

    # Grab unBique hit_ids and momenta of particle
    hit_ids = np.asarray(pdf.hit_id)
    xActual = np.asarray(pdf.tx)
    yActual = np.asarray(pdf.ty)
    zActual = np.asarray(pdf.tz)

    ########### Extract Nearby Points ###########
    # Assign to each hit in a track
    # a list of nearby opints
    #############################################
    xs = []
    ys = []
    zs = []
    nearbyDict = dict.fromkeys(range(len(xActual)))
    for key in nearbyDict:
        x1,y1,z1 = xActual[key], yActual[key], zActual[key]
        nearbyDF = find_nearby(dfHits,x1,y1,z1)
        for x in nearbyDF.x:
            xs.append(x)
        for y in nearbyDF.y:
            ys.append(y)
        for z in nearbyDF.z:
            zs.append(z)
            
    trace = go.Scatter3d(
        x = xs,
        y = ys,
        z = zs,
        mode = 'markers',
        name = 'Nearby',
        marker = dict(
        size = 3,
        color = 'orange'),
        )    
    
    return (trace)

    

def plot_characteristic_line(row):
    '''
    characteristic_line(row): given a row, plot the characteristic line of that track
    
    Input: 
    
    Return: an array populated with plotly graph_objs which store the information for 
            the characteristic line 
    '''
    
    part_id = dfP.loc[row,'particle_id']
    assert (part_id != 0), 'Hit due to noise, no associated track'
    pdf = dfT.loc[ dfT.particle_id == part_id ]

    # Grab unique hit_ids and momenta of particle
    hit_ids = np.asarray(pdf.hit_id)
    
    # First actual hit
    xActual_first = np.asarray(pdf.tx)[0]
    yActual_first = np.asarray(pdf.ty)[0]
    zActual_first = np.asarray(pdf.tz)[0]

    # Last Hit
    xActual_last = np.asarray(pdf.tx)[-1]
    yActual_last = np.asarray(pdf.ty)[-1]
    zActual_last = np.asarray(pdf.tz)[-1]

    # Vector of [origin, first hit, last hit]
#     xs = [0, xActual_first, xActual_last]
#     ys = [0, yActual_first, yActual_last]
#     zs = [0, zActual_first, zActual_last]
    
    xs = [0, xActual_last]
    ys = [0, yActual_last]
    zs = [0, zActual_last]

    ##############################
    # Plot the particle trajectory
    ##############################
    trace1 = go.Scatter3d(
        x = xs,
        y = ys,
        z = zs,
        mode = 'lines+markers',
        name = 'Characteristic Line',
        marker=dict(size=4,
                   color = 'purple'),
    )
    return (trace1)



def find_rd_ri(row):
    '''
    find_rd_ri(row): given a row, this function returns the solid angle of the 
                      corresponding track.
    
    Input : an integer representing a row number in either the truth file
    
    Return: the solid angle of the track corresponding to row (in sterradians?) 
    '''
    part_id = dfP.loc[row,'particle_id']
    assert (part_id != 0), 'Hit due to noise, no associated track'
    pdf = dfT.loc[ dfT.particle_id == part_id ]

    # Grab unique hit_ids and momenta of particle
    xActual = np.asarray(pdf.tx)
    yActual = np.asarray(pdf.ty)
    zActual = np.asarray(pdf.tz)
    
    maxrd = np.zeros(3)
    maxri = np.zeros(3)
    for i in range(len(xActual)):
        ri = [xActual[i], yActual[i], zActual[i]]
        rell = [xActual[-1], yActual[-1], zActual[-1]] 
        rd = ri - ( np.dot(ri, rell) / np.dot(rell,rell) ) * np.asarray(rell)
            
#         print ("r_ell dot r_d: {0}".format(np.dot(rell,rd)))
        assert (np.dot(rell,rd) <= np.abs(1)), "For hit {0}, r_ell and r_d are not quite perpindicular".format(i)

        if (np.dot(rd,rd) > np.dot(maxrd,maxrd)):
            maxrd = rd
            maxri = ri
    return (maxrd, maxri)  


def solid_angle(rd,ri):
    '''
    solid_angle(rd,ri):
    
    Input: rd is a 3-vector perpindicular rejection of the track point ri onto the characteristic line rell
           ri is a point on the track
           
    Output: The solid angle treating the length of rd as the effective area and  
            the length of ri as the distance away from the origin
    
    '''
    solid_angle = np.pi*np.sqrt(np.dot(rd,rd)) / np.sqrt( np.dot(ri,ri))
    return (solid_angle)

def plot_rd(row):
    '''
    plot_rd(row):
    
    Input:
    
    Output: plotly graph_objs representing the perpindicular rejection of ri onto rell at the point
            which maximizes distance between ri and rell
    '''
    rd, ri = find_rd_ri(row)
    
    trace1 = go.Scatter3d(
        x = [0,-rd[0]] + ri[0],
        y = [0,-rd[1]] + ri[1],
        z = [0,-rd[2]] + ri[2],
        mode = 'lines+markers',
        name = 'Radius',
        marker=dict(size=4,
                   color = 'green') 
    )
    return (trace1)

def plot_chart_attributes(row):
    part_id = dfP.loc[row,'particle_id']
    
    layout = go.Layout(
        title = 'Particle {0} Trajectory'.format(part_id),
        scene = dict(
            xaxis = dict(title = 'X Location [mm]'),
            yaxis = dict(title = 'Y Location [mm]'),
            zaxis = dict(title = 'Z Location [mm]'),
        )
    )
    return layout

print ('Complete')

Complete


## Load Data File

In [3]:
prefix = 'event000001000'
path = 'Input/train_100_events/'
dfHits, dfCells, dfParticles, dfTruth = load_event(path+prefix)

fileDict = {
    'Hits': dfHits,
    'Cells': dfCells,
    'Particles': dfParticles,
    'Truth': dfTruth
}

# for file in fileDict:
#     print ('File: {0} \n {1}'.format(file, fileDict[file].head()))
#     print ()
    
dfT = dfTruth
dfH = dfHits
dfP = dfParticles

print ('Complete')

Complete


## Plot a Track

In [4]:
row = 5229
track = plot_trajectory(row)

data = []
data.append(track)

fig = go.Figure(data=data)
offline.iplot(fig, filename='simple-3d-scatter')
print ('Complete')

Complete


## Plot the Start

In [5]:
row = 5229

start = plot_starting(row)
data = []
data.append(start)

fig = go.Figure(data = data)
offline.iplot(fig)
print ('Complete')

Complete


## Plot Nearby Points

In [6]:
row = 5229

near = plot_nearby(row)
data = []
data.append(near)

fig = go.Figure(data=data)
offline.iplot(fig)
print ('Complete')

Complete


## Plot Characteristic line

In [7]:
row = 5229
line = plot_characteristic_line(row)

data = [line]
fig = go.Figure(data=data)
offline.iplot(fig, filename='simple-3d-scatter')
print ('Complete')


Complete


## Plot $r_d$

In [8]:
row = 5229
rd = plot_rd(row)
data = [rd]
fig = go.Figure(data = data)
offline.iplot(fig)
print ('Complete')

Complete


## Plot Chart Attributes

In [9]:
row = 5229

layout = plot_chart_attributes(row)
fig = go.Figure(data = data,layout = layout)
offline.iplot(fig)


## Plotting Angular Size

My idea is:

1. Get a track
2. Plot the entire track
3. Stand at the origin and look out radially in the direction of the last point
4. Draw a circle which encompases all points within that track
5. This circle is probably related to the solid angle in some obvious way

### Qualitative Explanation
The idea is that I want to stand at the center of the collider, the origin, and look "in the direction" of the track. I call the "direction of the track" the _characteristic line_ which is the line pointing from the origin to the last point in the track. 

Once I am looking along this line, I want to draw a circle which encompasses all the points contained in that track. The size of this circle depends on how curvy the track is, and how close the curve is to  the origin. If the track bends sharply at the very beginning, the circle we draw will be very large because it is very close to our field of view. If the track bends sharply far away the size of the circle decreases as the curvy bits get farther away from the origin.

Here is a slightly more technical way of explaining what I want to do.

Given a track:
1. Look along a line in the general direction of the track.
2. This line defines a normal vector to some plane.
3. Project all points from the track onto this plane.
4. Draw a circle around all these points on this plane.



### Quantitative Discussion

I define the following quantities in the picture:

1. $\vec{r_\ell}$: this is the _characteristic line_ for a particular track
2. $\vec{r_i}$: this is the vector which points from the origin to a point on the track
3. $\vec{r_d}$: this is the perpindicular rejection of $\vec{r_i}$ onto $\vec{r_\ell}$

As a sanity check, we require that $\vec{r_\ell} \cdot \vec{r_d} = 0$
![](sketch.png)



The _characteristic line_ $\vec{r_\ell}$ the points in the general direction of the track. This is a quite naive approach though. A more sophisticated approach is to pick a line which minimizes the distance between itself and all points along the track.

We can write $$\vec{r_d} = \vec{r_i} - \text{Proj}_\vec{r_\ell} (\vec{r_i})$$
or, where $\text{Proj}_\vec{r_\ell} (\vec{r_i})$ is the projection of $\vec{r_i}$ onto $\vec{r_\ell}$.
This projection is given by:

$$ \text{Proj}_\vec{r_\ell} (\vec{r_i}) = (\vec{r}_i \cdot \vec{r}_\ell)\frac{\vec{r_\ell}}{|r_\ell|^2} $$

We then want to maximize the qantity $|\vec{r_d}|$. This represents the "farthest away" we have to look from the reference line. We do this by looping through all the points in the track $\vec{r_i}$ and calculate $|\vec{r_d}|$ for each point.

The maximum solid angle is then given by:
$$ \Omega = \frac{\pi|r_d|^2}{|r_i^2|} $$ at the point when $r_d$ is maximum.


## Plot Everything Together

In [18]:
row = 2991

rd,ri = find_rd_ri(row)
angle = solid_angle(rd,ri)
print ('Solid Angle: {0}'. format(angle))

track  = plot_trajectory(row)
start  = plot_starting(row)
line   = plot_characteristic_line(row)
rd     = plot_rd(row)
near   = plot_nearby(row)
layout = plot_chart_attributes(row)

data = []
data.append(track)
data.append(start)
data.append(line)
data.append(near)
data.append(rd)

fig = go.Figure(data = data, layout = layout)
offline.iplot(fig)


Solid Angle: 1.932484578841598


## Explanation:

The blue line is the track of the particle

The Red dot is the first registered hit of that particle which isn't necessarily $(0,0,0)$

The purple line is a line from the origin $(0,0,0)$ to the last hit in the particle track which I call the "characteristic line"

The green line is the point in the track which maximizes the solid angle.

If you stand at the origin (near the red dot) and look in the direction of the purple line (literally do this in the plot), a circle with radius equal to the green line will contain all the points for the track.

The solid angle is then calculated as:

$$ \Omega = \frac{\pi \ (\text{green line})^2}{(\text{distance to track-green line intersection})^2} $$